# CS246 - Colab 2
## Frequent Pattern Mining in Spark

In [3]:
from IPython.display import Image
print("Colab 2 Mascot")
Image(url='https://cdn.dribbble.com/users/222579/screenshots/1654898/stubby-ben-rex-roll.gif',width=150)

Colab 2 Mascot


### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [4]:
!pip install pyspark
!pip install -U -q PyDrive2
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 30 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 126315 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u442-b06~us1-0ubuntu1~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u442-b06~us1

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [6]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [9]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.kaggle.com/c/instacart-market-basket-analysis/data).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [10]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [11]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [12]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer).

In [13]:
''' 2 lines of code expected '''
# YOUR CODE HERE
df = orders.join(products, "product_id", "inner")
df = df.groupBy("order_id").agg(collect_list("product_name").alias("items"))
df.show()

+--------+--------------------+
|order_id|               items|
+--------+--------------------+
|       1|[Bulgarian Yogurt...|
|      96|[Roasted Turkey, ...|
|     112|[Fresh Cauliflowe...|
|     218|[Natural Artisan ...|
|     456|[Chorizo Pork, Pe...|
|     473|[Organic Whole Mi...|
|     631|[Organic Strawber...|
|     762|[Organic Strawber...|
|     774|[Ice Cream Variet...|
|     844|[Green Beans, Org...|
|     904|[Cup Noodles Chic...|
|     988|[Natural Vanilla ...|
|    1032|[Clover Org Greek...|
|    1077|[Bag of Organic B...|
|    1119|[Boneless Skinles...|
|    1139|[Banana, Organic ...|
|    1143|[Natural Premium ...|
|    1145|[Banana, Original...|
|    1275|[Boneless Skinles...|
|    1280|[Lactose Free Hal...|
+--------+--------------------+
only showing top 20 rows



In [14]:
df.dtypes

[('order_id', 'int'), ('items', 'array<string>')]

In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with

```minSupport=0.01``` and ```minConfidence=0.5```



In [18]:
''' 3 lines of code expected '''
# YOUR CODE HERE
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.01, minConfidence=0.5)
model = fpGrowth.fit(df)

Compute how many frequent itemsets and association rules were generated by running FP-growth alongside showing the table of top frequent itemsets and association rules.


In [19]:
''' 5 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
freq_itemsets = model.freqItemsets
num_freq_itemsets = freq_itemsets.count()
print(f"Number of frequent itemsets: {num_freq_itemsets}")

assoc_rules = model.associationRules
num_assoc_rules = assoc_rules.count()
print(f"Number of association rules: {num_assoc_rules}")

Number of frequent itemsets: 120
Number of association rules: 0


Now retrain the FP-growth model changing only
```minsupport=0.001```
and compute how many frequent itemsets and association rules were generated.


In [20]:
''' 5 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.001, minConfidence=0.5)
model = fpGrowth.fit(df)

freq_itemsets = model.freqItemsets
num_freq_itemsets = freq_itemsets.count()
print(f"Number of frequent itemsets: {num_freq_itemsets}")

assoc_rules = model.associationRules
num_assoc_rules = assoc_rules.count()
print(f"Number of association rules: {num_assoc_rules}")

Number of frequent itemsets: 4444
Number of association rules: 11


To conclude, go to Gradescope and read the remaining questions. We will ask you to inspect the resulting dataframes, and report a few results.



In [21]:
freq_itemsets.orderBy("freq", ascending=False).show(20, truncate=False)

+------------------------+-----+
|items                   |freq |
+------------------------+-----+
|[Banana]                |18726|
|[Bag of Organic Bananas]|15480|
|[Organic Strawberries]  |10894|
|[Organic Baby Spinach]  |9784 |
|[Large Lemon]           |8135 |
|[Organic Avocado]       |7409 |
|[Organic Hass Avocado]  |7293 |
|[Strawberries]          |6494 |
|[Limes]                 |6033 |
|[Organic Raspberries]   |5546 |
|[Organic Blueberries]   |4966 |
|[Organic Whole Milk]    |4908 |
|[Organic Cucumber]      |4613 |
|[Organic Zucchini]      |4589 |
|[Organic Yellow Onion]  |4290 |
|[Organic Garlic]        |4158 |
|[Seedless Red Grapes]   |4059 |
|[Asparagus]             |3868 |
|[Organic Grape Tomatoes]|3823 |
|[Organic Red Onion]     |3818 |
+------------------------+-----+
only showing top 20 rows



Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**! As a sanity check, for `minSupport=0.01` you should obtain `freq([Limes]) = 6033`